In [ ]:
also pls run this, I want to see if it's possible for us to develop code on our own machines using actually good IDEs then automatically download it to the robot from github

# Author: Claude Sonnet on April 30, 2025
import urllib.request
import os
import time

def download_github_file(url, save_path=None):
    """
    Download a file from GitHub using only built-in Python packages.
    
    Args:
        url (str): GitHub URL to the file
        save_path (str, optional): Path where to save the file. 
                                  If None, saves to current directory with original filename.
    
    Returns:
        str: Path to the downloaded file
    """
    # Convert GitHub web URL to raw content URL
    # Example: https://github.com/user/repo/blob/branch/path/file.ext
    # Becomes: https://raw.githubusercontent.com/user/repo/branch/path/file.ext
    if '/blob/' in url:
        raw_url = url.replace('github.com', 'raw.githubusercontent.com').replace('/blob/', '/')
    else:
        raise ValueError("URL doesn't appear to be a GitHub file URL")
    
    # Get filename from URL
    filename = url.split('/')[-1]
    
    # Set save path
    if save_path is None:
        save_path = filename
    
    print(f"Downloading from: {raw_url}")
    
    try:
        # Set up request with a user agent to avoid potential GitHub blocks
        headers = {'User-Agent': 'Python-Robot-Test/1.0'}
        req = urllib.request.Request(raw_url, headers=headers)
        
        # Download the file with a timeout of 10 seconds
        with urllib.request.urlopen(req, timeout=10) as response:
            content = response.read()
            
        # Save the content to the specified path
        with open(save_path, 'wb') as f:
            f.write(content)
            
        print(f"Successfully downloaded file to: {save_path}")
        return save_path
    
    except urllib.error.HTTPError as e:
        print(f"HTTP Error: {e.code} - {e.reason}")
        return None
    except urllib.error.URLError as e:
        print(f"URL Error: {e.reason}")
        return None
    except Exception as e:
        print(f"Unexpected error: {str(e)}")
        return None

def verify_download(file_path):
    """
    Verify that the file was downloaded successfully.
    
    Args:
        file_path (str): Path to the downloaded file
    
    Returns:
        bool: True if file exists and has content, False otherwise
    """
    if not os.path.exists(file_path):
        print(f"Error: File {file_path} does not exist")
        return False
    
    file_size = os.path.getsize(file_path)
    if file_size == 0:
        print(f"Error: File {file_path} is empty (0 bytes)")
        return False
    
    print(f"File {file_path} exists and has size: {file_size} bytes")
    
    # Print first few lines of the file
    try:
        with open(file_path, 'r') as f:
            first_lines = ''.join([next(f) for _ in range(5)])
        print(f"File preview:\n{first_lines}...")
        return True
    except Exception as e:
        print(f"Error reading file: {str(e)}")
        return False

def test_github_download():
    """
    Test the GitHub download functionality.
    """
    github_url = "https://github.com/pre-commit/demo-repo/blob/main/README.md"
    download_path = "downloaded_readme.md"
    
    print(f"Testing GitHub download capability...")
    print(f"URL: {github_url}")
    
    # Record start time
    start_time = time.time()
    
    # Attempt to download the file
    downloaded_file = download_github_file(github_url, download_path)
    
    # Calculate elapsed time
    elapsed_time = time.time() - start_time
    print(f"Download attempt completed in {elapsed_time:.2f} seconds")
    
    # Verify the download was successful
    if downloaded_file:
        success = verify_download(downloaded_file)
        if success:
            print("SUCCESS: Robot can download files from GitHub!")
            return True
        else:
            print("ERROR: File download appears to be incomplete or corrupt")
            return False
    else:
        print("ERROR: Failed to download file from GitHub")
        return False

if _name_ == "_main_":
    # Run the test
    test_github_download()